In [1]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('rugby_penalties_2019.csv')

In [9]:
df.head()

,Unnamed: 0,rounds,team_one,team_two,team_one_score,team_two_score,referee,venue,team_one_penalties,team_two_penalties,team_one_reds,team_two_reds,team_one_yellows,team_two_yellows
0,0,Round 15,Sunwolves,Rebels,7,52,Nick Briant,Prince Chichibu Memorial Stadium,14,10,0,0,0,1
1,1,Round 15,Sharks,Lions,27,17,Marius van der westhuizen,Jonsson Kings Park,8,8,0,0,0,0
2,2,Round 12,Hurricanes,Rebels,29,19,Nic Berry,Westpac Stadium,12,3,0,0,0,0
3,3,Round 11,Crusaders,Lions,36,10,Paul Williams,Christchurch Stadium,10,11,0,0,0,0
4,4,Round 18,Jaguares,Sunwolves,52,10,Federico Anselmi,Estadio Jose Amalfitani,7,12,0,0,0,1


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124 entries, 0 to 123
Data columns (total 14 columns):
Unnamed: 0            124 non-null int64
rounds                124 non-null object
team_one              124 non-null object
team_two              124 non-null object
team_one_score        124 non-null object
team_two_score        124 non-null object
referee               124 non-null object
venue                 124 non-null object
team_one_penalties    124 non-null object
team_two_penalties    124 non-null object
team_one_reds         124 non-null object
team_two_reds         124 non-null object
team_one_yellows      124 non-null object
team_two_yellows      124 non-null object
dtypes: int64(1), object(13)
memory usage: 13.6+ KB


In [16]:
df['team_one_penalties'] = pd.to_numeric(df['team_one_penalties'], errors='coerce')
df['team_two_penalties'] = pd.to_numeric(df['team_two_penalties'], errors='coerce')
df['penalty_difference'] = df['team_one_penalties'] - df['team_two_penalties']

In [18]:
def remove_neg(value):
    if value < 0:
        value = value * -1
        return value
    else:
        return value

In [20]:
df['penalty_difference'] = df['penalty_difference'].apply(remove_neg)

In [21]:
df['penalty_difference'].mean()

3.683333333333333

In [22]:
df[df['team_one'] == 'Rebels']

,Unnamed: 0,rounds,team_one,team_two,team_one_score,team_two_score,referee,venue,team_one_penalties,team_two_penalties,team_one_reds,team_two_reds,team_one_yellows,team_two_yellows,penalty_difference
6,6,Round 16,Rebels,NSW Waratahs,15,20,Paul Williams,AAMI Park,7.0,7.0,0,0,0,0,0.0
18,18,Round 4,Rebels,Brumbies,29,26,AJ Jacobs,AAMI Park,12.0,12.0,0,0,2,0,0.0
19,19,Round 9,Rebels,Stormers,24,41,Michael Fraser,AAMI Park,7.0,8.0,0,0,0,0,1.0
36,36,Round 18,Rebels,Chiefs,8,59,Angus Gardner,AAMI Park,7.0,8.0,0,0,0,0,1.0
37,37,Round 13,Rebels,Reds,30,24,Angus Gardner,AAMI Park,10.0,14.0,0,0,0,2,4.0
42,42,Round 8,Rebels,Sunwolves,42,15,Nic Berry,AAMI Park,16.0,6.0,0,0,2,1,10.0
102,102,Round 14,Rebels,Bulls,17,32,Federico Anselmi,AAMI Park,7.0,11.0,0,0,0,0,4.0
119,119,Round 3,Rebels,Highlanders,24,19,AJ Jacobs,AAMI Park,10.0,8.0,0,0,0,0,2.0


In [23]:
df.sort_values('team_one_penalties', ascending=False)['team_one_penalties'].mean()

8.775

In [25]:
df['team_two_penalties'].sum() - df['team_one_penalties'].sum()

34.0

In [27]:
df['penalty_difference'].groupby(df['referee']).mean()

referee
AJ Jacobs                     3.375000
Angus Gardner                 2.700000
Ben O'keeffe                  4.000000
Brendon Pickerill             4.250000
Damon Murphy                  4.666667
Egon Seconds                 10.000000
Federico Anselmi              3.571429
Glen Jackson                  3.600000
Jaco Peyper                   2.666667
Marius van der westhuizen     2.500000
Michael Fraser                4.100000
Nic Berry                     4.625000
Nick Briant                   4.250000
None                               NaN
Paul Williams                 1.333333
Rasta Rasivhenge              4.428571
Name: penalty_difference, dtype: float64

In [29]:
df['penalty_difference'].groupby(df['referee']).sum()

referee
AJ Jacobs                    27.0
Angus Gardner                27.0
Ben O'keeffe                 36.0
Brendon Pickerill            34.0
Damon Murphy                 28.0
Egon Seconds                 30.0
Federico Anselmi             25.0
Glen Jackson                 36.0
Jaco Peyper                  24.0
Marius van der westhuizen    20.0
Michael Fraser               41.0
Nic Berry                    37.0
Nick Briant                  34.0
None                          0.0
Paul Williams                12.0
Rasta Rasivhenge             31.0
Name: penalty_difference, dtype: float64

In [30]:
df['penalty_difference'].groupby(df['referee']).count()

referee
AJ Jacobs                     8
Angus Gardner                10
Ben O'keeffe                  9
Brendon Pickerill             8
Damon Murphy                  6
Egon Seconds                  3
Federico Anselmi              7
Glen Jackson                 10
Jaco Peyper                   9
Marius van der westhuizen     8
Michael Fraser               10
Nic Berry                     8
Nick Briant                   8
None                          0
Paul Williams                 9
Rasta Rasivhenge              7
Name: penalty_difference, dtype: int64

In [32]:
def get_nationality(value):
    
    if value == 'Blues':
        national = 'NZ'
    elif value == 'Brumbies':
        national = 'AUS'
    elif value == 'Bulls':
        national = 'RSA'
    elif value == 'Cheetahs':
        national = 'RSA'
    elif value == 'Chiefs':
        national = 'NZ'
    elif value == 'Crusaders':
        national = 'NZ'
    elif value == 'Force':
        national = 'AUS'
    elif value == 'Highlanders':
        national = 'NZ'
    elif value == 'Hurricanes':
        national = 'NZ'
    elif value == 'Jaguares':
        national = 'ARG'
    elif value == 'Kings':
        national = 'RSA'
    elif value == 'Lions':
        national = 'RSA'
    elif value == 'Rebels':
        national = 'AUS'
    elif value == 'Reds':
        national = 'AUS'
    elif value == 'Sharks':
        national = 'RSA'
    elif value == 'Stormers':
        national = 'RSA'
    elif value == 'Sunwolves':
        national = 'JAP'
    elif value == 'NSW Waratahs':
        national = 'AUS'
    
    return national

In [36]:
df['nationality_one'] = list(df.reset_index()['team_one'].apply(get_nationality).reset_index()['team_one'])
df['nationality_two'] = list(df.reset_index()['team_two'].apply(get_nationality).reset_index()['team_two'])

In [37]:
df

,Unnamed: 0,rounds,team_one,team_two,team_one_score,team_two_score,referee,venue,team_one_penalties,team_two_penalties,team_one_reds,team_two_reds,team_one_yellows,team_two_yellows,penalty_difference,nationality,nationality_one,nationality_two
0,0,Round 15,Sunwolves,Rebels,7,52,Nick Briant,Prince Chichibu Memorial Stadium,14.0,10.0,0,0,0,1,4.0,JAP,JAP,AUS
1,1,Round 15,Sharks,Lions,27,17,Marius van der westhuizen,Jonsson Kings Park,8.0,8.0,0,0,0,0,0.0,RSA,RSA,RSA
2,2,Round 12,Hurricanes,Rebels,29,19,Nic Berry,Westpac Stadium,12.0,3.0,0,0,0,0,9.0,NZ,NZ,AUS
3,3,Round 11,Crusaders,Lions,36,10,Paul Williams,Christchurch Stadium,10.0,11.0,0,0,0,0,1.0,NZ,NZ,RSA
4,4,Round 18,Jaguares,Sunwolves,52,10,Federico Anselmi,Estadio Jose Amalfitani,7.0,12.0,0,0,0,1,5.0,ARG,ARG,JAP
5,5,Round 18,Highlanders,NSW Waratahs,49,12,Michael Fraser,Rugby Park,4.0,7.0,0,0,0,1,3.0,NZ,NZ,AUS
6,6,Round 16,Rebels,NSW Waratahs,15,20,Paul Williams,AAMI Park,7.0,7.0,0,0,0,0,0.0,AUS,AUS,AUS
7,7,Round 4,Hurricanes,Highlanders,25,22,Damon Murphy,Westpac Stadium,9.0,12.0,0,0,0,0,3.0,NZ,NZ,NZ
8,8,Round 4,Crusaders,Chiefs,57,28,Brendon Pickerill,Christchurch Stadium,11.0,5.0,0,0,0,0,6.0,NZ,NZ,NZ
9,9,Round 11,Sunwolves,Highlanders,0,52,Ben O'keeffe,Prince Chichibu Memorial Stadium,10.0,9.0,0,0,0,0,1.0,JAP,JAP,NZ
